In [1]:
import boto3
import json
import datetime
import base64
import randomb
from botocore.config import Config

In [46]:
my_list = ['A','B','C','D'] 
city= (random.choices(my_list,weights=[4,34,34,28],k=1))
region=''
if(city[0]=='A'):
    region = 'ap-south-1'
elif(city[0]=='B'):
    region = 'us-east-1'
elif(city[0]=='C'):
    region = 'eu-west-1'
elif(city[0]=='D'):
    region = 'us-west-2'

config = Config(
retries = {
  'max_attempts': 3,
  'mode': 'standard'
}
)

region = 'ap-south-1'

client=boto3.client('rekognition',region, config=config)

In [3]:
from PIL import Image


In [66]:
def compare_faces(client,sourceFile,targetFile,threshold):
#     imgdata = base64.b64decode(str(sourceFile))
#     tgtdata = base64.b64decode(str(targetFile))
    
    response=client.compare_faces(SimilarityThreshold=int(threshold),
                                  SourceImage={'Bytes': sourceFile},
                                  TargetImage={'Bytes': targetFile})
    if len(response['FaceMatches'])==0:
        print('error no match')
        s3_path = add_to_S3(sourceFile,bucket_path)
        return {
             'status' : 'failure',
             'code'   : 'CD1004',
             's3Path' : str(s3_path) ,
             'ErrorMsg' : 'Face Not Matched'
            }
    else:        
        for faceMatch in response['FaceMatches']:
            similarity = faceMatch['Similarity']
            print(similarity)
            if float(similarity) < float(threshold) :
                s3_path =add_to_S3(sourceFile,bucket_path)
                return {
                'status' : 'failure',
                'code'   : 'CD1004',
                's3Path' : str(s3_path) ,
                'ErrorMsg' : 'Face Not Matched'
                }
            else:
                print("match")
#         print(response)

In [67]:
def detect_faces(client,photo):
    response = client.detect_faces(Image={"Bytes":photo})
    return len(response['FaceDetails'])

In [69]:
with open("/Users/raj/Documents/Rajat/ginger/ronal1.jpeg", "rb") as image_file:
    e1 = (image_file.read())
    
with open("/Users/raj/Documents/Rajat/ginger/ronal2.jpeg", "rb") as image_file:
    e2 = (image_file.read())
    
face_count=detect_faces(client,e1)
print(face_count)

if(face_count==1):
    compare_faces(client,e1,e2,90)
else:
    print("error more than one face detected")


1
99.99861907958984
match


In [70]:
## save to s3
def add_to_S3(image,bucket_path='aimlteam'):
    s3_client = boto3.client('s3')
    file_name  = str(datetime.datetime.now().timestamp())+".jpeg"
    fh = open("/tmp/"+file_name, "wb")
    fh.write(image)
    s3_client.upload_file("/tmp/"+file_name, bucket_path,file_name )
    fh.close()
    return bucket_path+"/"+file_name
